### Поиск LOV и его соседей

In [201]:
from bs4 import BeautifulSoup
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
import pandas as pd


In [202]:
with open("With_Cys/Cys_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

sequences = soup.sequences.findAll('sequence')

In [203]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])])

In [204]:
with open('With_Cys/final_proteins_4.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [205]:
new_seq = {}
for key in seqs.keys():
    if key in fasta:
        new_seq[key] = seqs[key]
seqs = new_seq

In [206]:
data = pd.read_csv('With_Cys/domains_4.csv')

In [207]:
data = data[data['seq id'].isin(seqs.keys())].reset_index(drop = True)

In [208]:
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,M0TGC4,170,266,162,267,PF13426.7,PAS_9,Domain,7,103,104,58.9,4.900000e-16,1,CL0183,Eukaryota Viridiplantae
1,M0TGC4,452,544,443,547,PF13426.7,PAS_9,Domain,9,101,104,73.1,1.900000e-20,1,CL0183,Eukaryota Viridiplantae
2,M0TGC4,633,918,630,918,PF00069.25,Pkinase,Domain,4,264,264,212.2,8.300000e-63,1,CL0016,Eukaryota Viridiplantae
3,M0KWF6,10,115,10,121,PF00072.24,Response_reg,Domain,1,105,112,70.0,1.800000e-19,1,CL0304,Archaea Euryarchaeota
4,M0KWF6,149,246,148,246,PF13426.7,PAS_9,Domain,2,104,104,39.1,7.000000e-10,1,CL0183,Archaea Euryarchaeota


In [214]:
data.shape

(5615, 16)

#### Находим LOV 

In [210]:
def IfLOV(row):
    b = False
    if row['clan'] == 'CL0183':
        for hit in seqs[row['seq id']]:
            if (int(row[3]) - hit[0]) <=5 and (hit[1] + 43 - int(row[4])) <= 1:
                return 'LOV'
    return row['hmm name']
        

In [211]:
data['hmm name'] = data.apply(IfLOV, axis = 1)

#### Находим Tandem LOV

In [212]:
from tqdm import tqdm

for name in tqdm(seqs.keys()):
    indexes = data[(data['seq id'] == name) & (data['hmm name'] == 'LOV')].index.values
    for i in indexes[:-1]:
        if (data.loc[i+1, 'hmm name'] == 'LOV' and 
            int(data.loc[i+1, 'envelope start']) - int(data.loc[i, 'envelope end']) <= 125):
            data.loc[i, 'envelope end'] = data.loc[i+1, 'envelope end']
            data.loc[i, 'hmm name'], data.loc[i+1, 'hmm name'] = 'TLOV', '_LOV'

100%|██████████| 1737/1737 [00:01<00:00, 875.63it/s]


In [213]:
data = data.loc[data['hmm name'] != '_LOV']

Отсеяла все лишние ловы (те которые входят вторыми в TLOV) теперь необходимо записать получившееся в отдельный файл, и уже теперь разбивать по царствам

In [215]:
tfile = open('With_Cys/domains_LOV_4.csv', 'a')
tfile.write(data.to_csv())
tfile.close()

### Primary effector

Пока без проверки еще и в Interpro так как запросы туда достаточно длительны + PAS domain там разбит на PAS и PAC. В будущем подделаю это

#### Бактерии

In [223]:
data_b = pd.read_csv('bacteria.csv')

In [224]:
data_b.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A0J6VGA8,62,156,52,157,PF13426.7,LOV,Domain,9,103,104,69.7,2.100000e-19,1,CL0183,Bacteria Proteobacteria
1,A0A0J6VGA8,175,235,173,237,PF00512.25,HisKA,Domain,3,64,67,34.0,2.200000e-08,1,CL0025,Bacteria Proteobacteria
2,A0A0J6VGA8,280,402,279,405,PF02518.26,HATPase_c,Domain,2,109,112,49.2,6.500000e-13,1,CL0025,Bacteria Proteobacteria
3,A0A0J6VGA8,427,537,427,538,PF00072.24,Response_reg,Domain,1,111,112,67.4,1.100000e-18,1,CL0304,Bacteria Proteobacteria
4,A0A098T9D3,47,149,46,150,PF13426.7,LOV,Domain,2,103,104,61.8,6.100000e-17,1,CL0183,Bacteria Proteobacteria


In [225]:
names = set(data_b['seq id']) #множество всех имен

In [226]:
prim_eff_bac = {}

In [227]:
from tqdm import tqdm
for name in tqdm(names):
    indexes = data_b[data_b['seq id'] == name].index.values
    for i in indexes:
        if data_b.loc[i, 'hmm name'] == 'LOV':
            if (i < indexes[-1] and 
                int(data_b.loc[i+1, 'envelope start']) - int(data_b.loc[i, 'envelope end']) <= 125 and
                float(data_b.loc[i+1, 'E-value']) <= 1e-3):
                n = prim_eff_bac.setdefault(data_b.loc[i+1, 'hmm name'], 0) 
                n += 1
                prim_eff_bac[data_b.loc[i+1, 'hmm name']] = n
            elif (i > indexes[0] and 
                  int(data_b.loc[i, 'envelope start']) - int(data_b.loc[i-1, 'envelope end']) <= 125 and 
                  float(data_b.loc[i-1, 'E-value']) <= 1e-3):
                n = prim_eff_bac.setdefault(data_b.loc[i-1, 'hmm name'], 0) 
                n += 1
                prim_eff_bac[data_b.loc[i-1, 'hmm name']] = n
            else:
                n = prim_eff_bac.setdefault('No domain', 0) 
                n += 1
                prim_eff_bac['No domain'] = n

100%|██████████| 4015/4015 [00:03<00:00, 1076.48it/s]


In [228]:
prim_eff_bac

{'ANTAR': 1,
 'EAL': 1,
 'GAF': 70,
 'GAF_2': 45,
 'GAF_3': 7,
 'GGDEF': 496,
 'GerE': 161,
 'Guanylate_cyc': 4,
 'HATPase_c': 41,
 'HATPase_c_2': 5,
 'HTH_18': 4,
 'HWE_HK': 157,
 'HisKA': 427,
 'HisKA_2': 324,
 'HisKA_3': 4,
 'MASE1': 17,
 'MCPsignal': 10,
 'No domain': 1451,
 'PAS': 87,
 'PAS_3': 267,
 'PAS_4': 66,
 'PAS_7': 3,
 'PAS_8': 51,
 'PAS_9': 89,
 'Response_reg': 3,
 'SBP_bac_3': 1,
 'STAS': 204,
 'SpoIIE': 16,
 'Trans_reg_C': 1}

#### Археи

In [229]:
data_a = pd.read_csv('archaea.csv')

In [230]:
data_a[data_a['hmm name'] == 'TLOV']

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom


In [231]:
prim_eff_arch = {}

In [232]:
from tqdm import tqdm

for name in tqdm(set(data_a['seq id'])):
    indexes = data_a[data_a['seq id'] == name].index.values
    for i in indexes:
        if data_a.loc[i, 'hmm name'] == 'LOV':
            if (i < indexes[-1] and 
                int(data_a.loc[i+1, 'envelope start']) - int(data_a.loc[i, 'envelope end']) <= 125 and
                float(data_a.loc[i+1, 'E-value']) <= 1e-3):
                n = prim_eff_arch.setdefault(data_a.loc[i+1, 'hmm name'], 0) 
                n += 1
                prim_eff_arch[data_a.loc[i+1, 'hmm name']] = n
            elif (i > indexes[0] and 
                  int(data_a.loc[i, 'envelope start']) - int(data_a.loc[i-1, 'envelope end']) <= 125 and 
                  float(data_a.loc[i-1, 'E-value']) <= 1e-3):
                n = prim_eff_arch.setdefault(data_a.loc[i-1, 'hmm name'], 0) 
                n += 1
                prim_eff_arch[data_a.loc[i-1, 'hmm name']] = n
            else:
                n = prim_eff_arch.setdefault('No domain', 0) 
                n += 1
                prim_eff_arch['No domain'] = n

100%|██████████| 362/362 [00:00<00:00, 1990.22it/s]


In [233]:
prim_eff_arch

{'GAF': 1,
 'GAF_2': 92,
 'GAF_3': 1,
 'HATPase_c': 30,
 'HisKA': 55,
 'No domain': 53,
 'PAS': 23,
 'PAS_3': 6,
 'PAS_4': 58,
 'PAS_8': 5,
 'PAS_9': 36,
 'Response_reg': 1}

#### Грибы

In [234]:
data_f = pd.read_csv('fungi.csv')

In [235]:
prim_eff_fung = {}

In [236]:
from tqdm import tqdm

for name in tqdm(set(data_f['seq id'])):
    indexes = data_f[data_f['seq id'] == name].index.values
    for i in indexes:
        if data_f.loc[i, 'hmm name'] == 'LOV':
            if (i < indexes[-1] and 
                int(data_f.loc[i+1, 'envelope start']) - int(data_f.loc[i, 'envelope end']) <= 125 and
                float(data_f.loc[i+1, 'E-value']) <= 1e-3):
                n = prim_eff_fung.setdefault(data_f.loc[i+1, 'hmm name'], 0) 
                n += 1
                prim_eff_fung[data_f.loc[i+1, 'hmm name']] = n
            elif (i > indexes[0] and 
                  int(data_f.loc[i, 'envelope start']) - int(data_f.loc[i-1, 'envelope end']) <= 125 and 
                  float(data_f.loc[i-1, 'E-value']) <= 1e-3):
                n = prim_eff_fung.setdefault(data_f.loc[i-1, 'hmm name'], 0) 
                n += 1
                prim_eff_fung[data_f.loc[i-1, 'hmm name']] = n
            else:
                n = prim_eff_fung.setdefault('No domain', 0) 
                n += 1
                prim_eff_fung['No domain'] = n

100%|██████████| 1148/1148 [00:00<00:00, 1890.13it/s]


In [237]:
prim_eff_fung

{'Cupin_2': 2,
 'GGDEF': 1,
 'No domain': 529,
 'PAS': 36,
 'PAS_11': 1,
 'PAS_3': 488,
 'PAS_4': 2,
 'PAS_8': 1,
 'RGS': 88}

#### Протисты

In [238]:
data_pr = pd.read_csv('protista.csv')

In [239]:
prim_eff_prot = {}

In [242]:
from tqdm import tqdm

for name in tqdm(set(data_pr['seq id'])):
    indexes = data_pr[data_pr['seq id'] == name].index.values
    for i in indexes:
        if data_pr.loc[i, 'hmm name'] == 'LOV':
            if (i < indexes[-1] and 
                int(data_pr.loc[i+1, 'envelope start']) - int(data_pr.loc[i, 'envelope end']) <= 125 and
                float(data_pr.loc[i+1, 'E-value']) <= 1e-3):
                n = prim_eff_prot.setdefault(data_pr.loc[i+1, 'hmm name'], 0) 
                n += 1
                prim_eff_prot[data_pr.loc[i+1, 'hmm name']] = n
            elif (i > indexes[0] and 
                  int(data_pr.loc[i, 'envelope start']) - int(data_pr.loc[i-1, 'envelope end']) <= 125 and 
                  float(data_pr.loc[i-1, 'E-value']) <= 1e-3):
                n = prim_eff_prot.setdefault(data_pr.loc[i-1, 'hmm name'], 0) 
                n += 1
                prim_eff_prot[data_pr.loc[i-1, 'hmm name']] = n
            else:
                n = prim_eff_prot.setdefault('No domain', 0) 
                n += 1
                prim_eff_prot['No domain'] = n

100%|██████████| 180/180 [00:00<00:00, 2125.54it/s]


In [243]:
prim_eff_prot

{'No domain': 67, 'RGS': 4, 'SAM_2': 3, 'bZIP_1': 21, 'bZIP_2': 86}

#### Растения

In [244]:
data_p = pd.read_csv('plants.csv')

In [245]:
prim_eff_pl = {}

In [248]:
from tqdm import tqdm

for name in tqdm(set(data_p['seq id'])):
    indexes = data_p[data_p['seq id'] == name].index.values
    for i in indexes:
        if data_p.loc[i, 'hmm name'] == 'LOV':
            if (i < indexes[-1] and 
                int(data_p.loc[i+1, 'envelope start']) - int(data_p.loc[i, 'envelope end']) <= 125 and
                float(data_p.loc[i+1, 'E-value']) <= 1e-3):
                n = prim_eff_pl.setdefault(data_p.loc[i+1, 'hmm name'], 0) 
                n += 1
                prim_eff_pl[data_p.loc[i+1, 'hmm name']] = n
            elif (i > indexes[0] and 
                  int(data_p.loc[i, 'envelope start']) - int(data_p.loc[i-1, 'envelope end']) <= 125 and 
                  float(data_p.loc[i-1, 'E-value']) <= 1e-3):
                n = prim_eff_pl.setdefault(data_p.loc[i-1, 'hmm name'], 0) 
                n += 1
                prim_eff_pl[data_p.loc[i-1, 'hmm name']] = n
            else:
                n = prim_eff_pl.setdefault('No domain', 0) 
                n += 1
                prim_eff_pl['No domain'] = n

100%|██████████| 4421/4421 [00:04<00:00, 975.01it/s]


In [249]:
prim_eff_pl

{'DUF3700': 1,
 'F-box': 441,
 'F-box-like': 630,
 'GAF': 1,
 'HisKA': 47,
 'Kelch_3': 1,
 'Kelch_4': 1,
 'Lipase_GDSL_2': 2,
 'No domain': 2791,
 'PAS_3': 4,
 'PAS_9': 204,
 'Pkinase': 1955,
 'RGS': 1}

###  Мультидоменные структуры

#### Археи

In [250]:
data_a.groupby(by = 'hmm name').groups

{'BAT': Int64Index([  18,   50,   54,   71,   87,  100,  104,  118,  138,  142,
             ...
             1559, 1567, 1583, 1593, 1597, 1600, 1613, 1617, 1626, 1648],
            dtype='int64', length=132),
 'GAF': Int64Index([10, 120, 177, 404, 450, 471, 504, 585, 810, 875, 879, 992, 1102,
             1528],
            dtype='int64'),
 'GAF_2': Int64Index([  17,   49,   59,   70,   76,   86,  103,  109,  117,  137,
             ...
             1507, 1522, 1549, 1558, 1582, 1592, 1621, 1625, 1637, 1647],
            dtype='int64', length=109),
 'GAF_3': Int64Index([448, 491, 549, 1157], dtype='int64'),
 'HATPase_c': Int64Index([   4,    9,   13,   24,   30,   35,   41,   47,   61,   68,
             ...
             1574, 1580, 1590, 1607, 1610, 1623, 1633, 1639, 1644, 1654],
            dtype='int64', length=195),
 'HATPase_c_2': Int64Index([403, 513, 605, 619, 631], dtype='int64'),
 'HTH_10': Int64Index([  19,   51,   55,   72,   88,  101,  105,  119,  139,  143,
             

In [274]:
from tqdm import tqdm
forward = {}
backward = {}
for name in tqdm(set(data_a['seq id'])):
    index = data_a[data_a['seq id'] == name].index.values
    LOV = data_a[(data_a['seq id'] == name) & (data_a['hmm name'] == 'LOV')].index.values
    for L in LOV:
        for i in range(L, index[-1]):
            if (int(data_a.loc[i+1, 'envelope start']) - int(data_a.loc[i, 'envelope end']) <= 125 and
                float(data_a.loc[i+1, 'E-value']) <= 1e-3):
                s = forward.setdefault(data_a.loc[i+1, 'hmm name'], [0 for i in range(8)]) 
                if (i+1 - L) < len(s):
                    s[i-L] += 1
                forward[data_a.loc[i+1, 'hmm name']] = s
            else:
                break
        for i in range(L, index[0], -1):
            if (int(data_a.loc[i, 'envelope start']) - int(data_a.loc[i-1, 'envelope end']) <= 125 and
                float(data_a.loc[i-1, 'E-value']) <= 1e-3):
                s = backward.setdefault(data_a.loc[i+1, 'hmm name'], [0 for i in range(8)]) 
                if (L - i + 1) < len(s):
                    s[L - i] += 1
                backward[data_a.loc[i+1, 'hmm name']] = s
            else:
                break

100%|██████████| 362/362 [00:00<00:00, 737.95it/s]


In [261]:
forward = {}
backward = {}
index = data_a[data_a['seq id'] == 'A0A0D6JLL4'].index.values
LOV = data_a[(data_a['seq id'] == 'A0A0D6JLL4') & (data_a['hmm name'] == 'LOV')].index.values
for L in LOV:
    for i in range(L, index[-1]):
        if (int(data_a.loc[i+1, 'envelope start']) - int(data_a.loc[i, 'envelope end']) <= 125 and
            float(data_a.loc[i+1, 'E-value']) <= 1e-3):
            s = forward.setdefault(data_a.loc[i+1, 'hmm name'], [0 for i in range(8)]) 
            if (i+1 - L) < len(s):
                s[i - L] += 1
            forward[data_a.loc[i+1, 'hmm name']] = s
        else:
            break
    for i in range(L, index[0], -1):
        if (int(data_a.loc[i, 'envelope start']) - int(data_a.loc[i-1, 'envelope end']) <= 125 and
            float(data_a.loc[i-1, 'E-value']) <= 1e-3):
            s = backward.setdefault(data_a.loc[i+1, 'hmm name'], [0 for i in range(8)]) 
            if (L - i + 1) < len(s):
                s[L - i] += 1
            backward[data_a.loc[i+1, 'hmm name']] = s
        else:
            break

In [275]:
forward

{'BAT': [0, 74, 0, 1, 0, 1, 2, 0],
 'GAF': [1, 0, 1, 0, 0, 0, 0, 0],
 'GAF_2': [92, 3, 1, 1, 3, 2, 0, 0],
 'GAF_3': [1, 0, 0, 0, 0, 0, 0, 0],
 'HATPase_c': [30, 70, 59, 28, 4, 0, 0, 0],
 'HATPase_c_2': [0, 3, 0, 1, 1, 0, 0, 0],
 'HTH_10': [0, 0, 74, 0, 1, 0, 1, 0],
 'HisKA': [55, 53, 6, 2, 0, 0, 0, 0],
 'PAS': [19, 2, 6, 2, 1, 1, 0, 0],
 'PAS_3': [5, 32, 6, 2, 0, 0, 0, 0],
 'PAS_4': [52, 10, 20, 3, 0, 0, 1, 0],
 'PAS_8': [0, 2, 1, 0, 0, 0, 0, 0],
 'PAS_9': [31, 1, 1, 0, 0, 0, 0, 0]}

### Графики 

In [280]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib

Using matplotlib backend: TkAgg


In [279]:
fig, (a, b, f, pr, pl) = plt.subplots(1, 5, sharey=True)
ind = np.arrange(19)

In [ ]:
a.barh(ind, )